In [2]:
# 数据处理
import pandas as pd
import numpy as np
import json
import csv
import random
import math
from itertools import product


# 网络分析与可视化
import networkx as nx
from pyvis import network as net
import matplotlib.pyplot as plt
import powerlaw # Power laws are probability distributions with the form:p(x)∝x−α

In [3]:
# 读取已经处理好的完整语义网络数据

df_complete = pd.read_csv('01_Processed Data/Complete-Data.csv')
df_complete.head()

,Concept,Rel,Feature,Frequency,familiarity,concretness,上级类别,下级类别,coverage,cue_validity,categorical_distinctiveness
0,安全带,可以,可以-保护,27.096774,9.0,9.0,人造物,工具,0.851345,0.038320,0.158537
1,安全带,是,是-带子,22.258065,9.0,9.0,人造物,工具,0.851345,0.286019,0.012195
2,安全带,其他,其他-车,17.419355,9.0,9.0,人造物,工具,0.851345,0.122012,0.021341
3,安全带,是,是-安保的,16.451613,9.0,9.0,人造物,工具,0.851345,0.089938,0.041159
4,安全带,可以,可以-保护人,15.483871,9.0,9.0,人造物,工具,0.851345,0.149063,0.035061


In [8]:
with open('02_Graph/MasterConceptNetwork.json') as f:
    js_graph = json.load(f)
    G_MasterConceptNetwork = nx.json_graph.node_link_graph(js_graph)

In [14]:
G_MasterConceptNetwork.nodes['蕨菜']

{'subcategory': '蔬菜',
 'maincategory': '植物',
 'closeness_centrality': 0.09507346585998272,
 'clustering_coefficent': 0.4936462,
 'degree_centrality': 0.09220779220779221,
 'betweenness_centrality': 0,
 'pagerank': 0.0020449022591244206,
 'laplacian_centrality': 0.018391013697218506,
 'louvain_community': 4,
 'x': -119.952515,
 'y': -662.04565}

In [16]:
G_MasterConceptNetwork.nodes['青菜']

{'subcategory': '蔬菜',
 'maincategory': '植物',
 'closeness_centrality': 0.09035437690682939,
 'clustering_coefficent': 0.4955365,
 'degree_centrality': 0.09350649350649351,
 'betweenness_centrality': 0,
 'pagerank': 0.0021082399282271813,
 'laplacian_centrality': 0.018805097845421236,
 'louvain_community': 4,
 'x': -147.51857,
 'y': -763.1568}

In [4]:
# 将子图内的节点按重要性排序，以logspace采样出指定数量的节点
def NodeSampling(G,num_of_sample, mode):
    # pr = nx.pagerank(G, alpha=0.85,weight='weight')
    # words = list(dict(sorted(pr.items(), key=lambda item: item[1], reverse=True)))
    words = list(sorted(G.nodes(), key=lambda n: G.nodes[n]['closeness_centrality'], reverse=True))

    if mode == 'log':
        # np.logspace(start, end, num_of_samples, endpoint=True, base=10.0)
        index_list = np.logspace(0, math.log(len(words),10), num_of_sample, endpoint=True)
        index_list = set([int(x) for x in index_list])
        words = [words[i] for i in index_list]
    elif mode == 'top':
        words = words[0:num_of_sample]

    return words

# 查看全部节点在某个属性的所有选项
def NodeAttributeValueList(G,attribute):
    return set(np.array([G.nodes[n][attribute] for n in G.nodes]).flatten())

# 类别子图
def ClusterFilter(G,nodeAttribute, nodeValue, edgeAttribute, edgeValue, writeFile, format):

    # 筛选子图 Node & Edge Attribute Filter
    def filter_node(node):
        if G.nodes[node][nodeAttribute] == nodeValue:
            return node 

    def filter_edge(u,v):
        if G[u][v][edgeAttribute] > edgeValue:
            return G[u][v]

    view = nx.subgraph_view(G,filter_node=filter_node, filter_edge=filter_edge,)
    largest_cc = max(nx.connected_components(view), key=len)
    view = view.subgraph(largest_cc)

    # 储存文件
    if writeFile == True:
        if format == 'Gephi':
            fileName = '02_Graph/ConceptNetwork_'+nodeAttribute+'_'+str(nodeValue)+'_'+edgeAttribute+str(edgeValue)+'.gexf'
            nx.write_gexf(view, fileName)

        else:
            fileName = '02_Graph/ConceptNetwork_'+nodeAttribute+'_'+str(nodeValue)+'_'+edgeAttribute+str(edgeValue)+'.json'
            # fileName = '02_Graph/Wenyue/ConceptNetwork_'+nodeAttribute+'_'+str(nodeValue)+'.json'
            with open(fileName,'w+') as f:
                if format == 'G6':
                    f.write(json.dumps(nx.node_link_data(view), ensure_ascii=False).replace('links','edges'))
                if format == 'D3':
                    f.write(json.dumps(nx.node_link_data(view), ensure_ascii=False))

    # 打印提示
    print("Number of Nodes:",len(view.nodes))
    # print("Node Degree Hist")
    # NodeDegreeHist(view)
    # print("Edge Weight Hist")
    # EdgeWeightHist(view)
    
    return view

In [ ]:
每个大category20词
按subcategory的总词数来比例分配词汇
选每个subcategory中最重要的一些词

In [33]:
cue_words = []

maincategorys = NodeAttributeValueList(G_MasterConceptNetwork,'maincategory')
for i in maincategorys:
    print(i)
    view = ClusterFilter(G_MasterConceptNetwork,'maincategory',i,'weight',0,True,'Gephi')
    # subcategorys = NodeAttributeValueList(view,'subcategory')
    # for j in subcategorys:
    #     nodes = [x for x,y in view.nodes(data=True) if y['subcategory']==j]
    #     print(j, len(nodes))
    cue_words_maincategory = NodeSampling(view,20,'log')
    cue_words = cue_words + cue_words_maincategory
    print(cue_words_maincategory)

植物
Number of Nodes: 207
['仙人掌', '花', '狗尾草', '菊花', '金银花', '竹子', '无花果', '风信子', '枇杷', '山楂', '泡菜', '青梅', '满天星', '桑葚', '青菜', '海藻', '脐橙', '萝卜']
交通工具
Number of Nodes: 42
['汽车', '吉普车', '摩托车', '消防车', '运钞车', '碰碰车', '赛车', '行李车', '帆船', '电动汽车', '公交车', '豪华轿车', '自行车', '有轨电车', '地铁']
人造物
Number of Nodes: 251
['盾牌', '眼镜盒', '卷帘', '枕套', '浴帘', '长袖', '墙纸', '礼服', '拖鞋', '床罩', '篱笆', '护腕', '运动装', '电池', '凉鞋', '镜框', '枪', '皮帽']
自然物
Number of Nodes: 7
['大理石', '珍珠', '石头', '水晶', '宝石', '钻石']
食物
Number of Nodes: 141
['白酒', '纯净水', '白水', '瓶装水', '冰红茶', '可口可乐', '葡萄酒', '汽水', '泉水', '蛋卷', '果汁', '冰棍', '橘子汽水', '糖浆', '草莓酱', '豆浆', '魔芋']
身体部位
Number of Nodes: 37
['脚背', '指甲', '眉毛', '泪珠', '骨架', '头皮', '头', '肋骨', '臀部', '拇指', '喉', '胡须', '手', '腿', '大脑']


In [46]:
G_MasterConceptNetwork.nodes()['脚背']['maincategory']

'身体部位'

In [47]:
concepts = []
maincategorys = []
subcategorys = []

for i in list(G_MasterConceptNetwork.nodes()):
    concepts.append(i)
    maincategorys.append(G_MasterConceptNetwork.nodes()[i]['maincategory'])
    subcategorys.append(G_MasterConceptNetwork.nodes()[i]['subcategory'])

df_nodes = pd.DataFrame(concepts,columns=["concept"])

In [54]:
df_nodes['maincategory'] = maincategorys
df_nodes['subcategory'] = subcategorys
df_nodes = df_nodes.sort_values(['maincategory','subcategory'])

In [57]:
df_nodes

,concept,maincategory,subcategory
158,独木舟,交通工具,船
170,帆船,交通工具,船
231,豪华客轮,交通工具,船
459,皮艇,交通工具,船
480,潜艇,交通工具,船
...,...,...,...
616,威士忌酒,食物,酒水饮料
624,乌龙茶,食物,酒水饮料
633,稀饭,食物,酒水饮料
640,香槟,食物,酒水饮料


In [56]:
df_nodes.to_csv('02_Graph/G_MasterConceptNetwork_nodes.csv',index=False)

In [ ]:
# df_ = pd.DataFrame(cue_words, columns=['cue_word'])
# df_ = Sample_PWA(df_,0.5,'picture_naming')
# df_.to_csv('02_Graph/Wenyue/PictureNamingResult.csv',index=False)

In [42]:
cue_words

['仙人掌',
 '花',
 '狗尾草',
 '菊花',
 '金银花',
 '竹子',
 '无花果',
 '风信子',
 '枇杷',
 '山楂',
 '泡菜',
 '青梅',
 '满天星',
 '桑葚',
 '青菜',
 '海藻',
 '脐橙',
 '萝卜',
 '汽车',
 '吉普车',
 '摩托车',
 '消防车',
 '运钞车',
 '碰碰车',
 '赛车',
 '行李车',
 '帆船',
 '电动汽车',
 '公交车',
 '豪华轿车',
 '自行车',
 '有轨电车',
 '地铁',
 '盾牌',
 '眼镜盒',
 '卷帘',
 '枕套',
 '浴帘',
 '长袖',
 '墙纸',
 '礼服',
 '拖鞋',
 '床罩',
 '篱笆',
 '护腕',
 '运动装',
 '电池',
 '凉鞋',
 '镜框',
 '枪',
 '皮帽',
 '大理石',
 '珍珠',
 '石头',
 '水晶',
 '宝石',
 '钻石',
 '白酒',
 '纯净水',
 '白水',
 '瓶装水',
 '冰红茶',
 '可口可乐',
 '葡萄酒',
 '汽水',
 '泉水',
 '蛋卷',
 '果汁',
 '冰棍',
 '橘子汽水',
 '糖浆',
 '草莓酱',
 '豆浆',
 '魔芋',
 '脚背',
 '指甲',
 '眉毛',
 '泪珠',
 '骨架',
 '头皮',
 '头',
 '肋骨',
 '臀部',
 '拇指',
 '喉',
 '胡须',
 '手',
 '腿',
 '大脑']

In [34]:
list_full_current = ['蕨菜',
 '青菜',
 '橄榄',
 '生菜',
 '番茄',
 '卷心菜',
 '花菜',
 '大头菜',
 '西兰花',
 '蓝莓',
 '茼蒿',
 '红薯',
 '花',
 '笋',
 '草',
 '仙人掌',
 '油麦菜',
 '莴笋',
 '草莓',
 '浮萍',
 '煎蛋卷',
 '奶酪',
 '牛奶',
 '肉丸',
 '饼干',
 '油饼',
 '蛋卷',
 '火腿肠',
 '奶油',
 '清汤',
 '火腿',
 '香肠',
 '甜点',
 '巧克力牛奶',
 '棉花糖',
 '粥',
 '馒头',
 '腊肠',
 '酸奶',
 '烤排骨',
 '刺猬',
 '鲤鱼',
 '猪',
 '海豹',
 '豚鼠',
 '狐狸',
 '鲸鱼',
 '海狮',
 '松鼠',
 '狼',
 '犀牛',
 '鲶鱼',
 '美洲狮',
 '猴子',
 '老虎',
 '羊驼',
 '网纹蟒',
 '熊',
 '鲫鱼',
 '黄鼠狼',
 '便盆',
 '漏斗',
 '清洁球',
 '针线篮',
 '桶',
 '陀螺',
 '扫帚',
 '猫砂',
 '簸箕',
 '管道',
 '储蓄罐',
 '沙漏',
 '万花筒',
 '骨灰盒',
 '积木',
 '地图',
 '滑轮',
 '排水管',
 '哑铃',
 '壁画',
 '汽车',
 '卡车',
 '吉普车',
 '赛车',
 '轿车',
 '跑车',
 '车辆',
 '豪华轿车',
 '公交车',
 '摩托车',
 '电动汽车',
 '卡丁车',
 '出租车',
 '三轮车',
 '巴士',
 '敞篷跑车',
 '房车',
 '自行车',
 '行李车',
 '手推车',
 '沟渠',
 '岩石',
 '水晶',
 '宝石',
 '月亮',
 '沙坑',
 '煤田',
 '大理石',
 '钻石',
 '雪花',
 '煤炭',
 '流星',
 '白银',
 '珍珠',
 '冰雹',
 '星星',
 '露珠',
 '沙尘暴'
 '石头',
 '雪球',
 '臀部',
 '肩膀',
 '脚趾',
 '脚背',
 '嘴唇',
 '拇指',
 '膝盖',
 '大腿',
 '心脏',
 '腿',
 '关节',
 '手臂',
 '脚后跟',
 '血管',
 '手指',
 '脚',
 '牙齿',
 '喉',
 '手掌',
 '睫毛']

In [37]:
list_delete_current = ['蕨菜',
 '橄榄',
 '生菜',
 '大头菜',
 '茼蒿',
 '花',
 '油麦菜',
 '浮萍',
 '煎蛋卷',
 '油饼',
 '奶油',
 '清汤',""
 '火腿',
 '甜点',
 '巧克力牛奶',
 '腊肠',
 '烤排骨',
 '鲤鱼',
 '豚鼠',
 '海狮',
 '鲶鱼',
 '羊驼',
 '网纹蟒',
 '鲫鱼',
 '黄鼠狼',
 '便盆',
 '清洁球',
 '猫砂',
 '簸箕',
 '管道',
 '万花筒',
 '骨灰盒',
 '地图',
 '滑轮',
 '排水管',
 '壁画',
 '汽车',
 '跑车',
 '车辆',
 '豪华轿车',
 '电动汽车',
 '卡丁车',
 '敞篷跑车',
 '房车',
 '行李车',
 '沟渠',
 '宝石',
 '沙坑',
 '煤田',
 '煤炭',
 '臀部',
 '肩膀',
 '脚趾',
 '脚背',
 '大腿',
 '关节',
 '脚后跟',
 '手指',
 '睫毛']

In [41]:
list_valid_current = list(set(list_full_current) ^ set(list_delete_current))

In [62]:
list_valid_next = list(set(concepts).intersection(set(list_valid_current)))

In [64]:
len(list_valid_next)

55

In [65]:
len(cue_words)

89

In [63]:
list(set(list_valid_next).intersection(set(cue_words)))

['大理石',
 '腿',
 '珍珠',
 '吉普车',
 '赛车',
 '青菜',
 '蛋卷',
 '公交车',
 '摩托车',
 '自行车',
 '拇指',
 '钻石',
 '仙人掌',
 '水晶',
 '喉']

In [74]:
list_checklist_next = set(cue_words).union(set(list_valid_next)).difference(set(list_delete_current))

In [75]:
df_nodes[df_nodes['concept'].isin(list_checklist_next)].to_csv("02_VASystemData/check_list.csv",index=False)

## 生成命名测试用数据 Picture Naming Test Data Generation

In [2]:
## Input 
word_list = ['豆浆', '菠萝汁', '咖啡']


# word_list = cue_words

data = {
    'currentStep': 0,
    'steps': []
}

for word in word_list:
    new_step = {
        'name':word,
        # 'image':'local-resource:///Users/zhanghexin/aphasia_viz/src/assets/test_images/'+word+'.jpeg',
        'image':'/test_images/'+word+'.jpeg',
        'countdown':20,
        'result':'fail',
        'status':'unchecked',
    }
    data['steps'].append(new_step)

with open("02_VASystemData/test_picture-naming.json", "w") as f: 
    json.dump(data, f, ensure_ascii=False)

## 生成语义特征分析训练用数据 SFA Data Generation

In [9]:
 #列出共有某个语义特征的同类概念

def List_Related_Concepts(concept,feature):
    main_category = df_complete[df_complete['Concept'] == concept]['上级类别'].unique()[0]
    related_concepts = list(df_complete[(df_complete['上级类别'] == main_category) & (df_complete['Feature'] == feature)]['Concept'].unique())
    related_concepts = list(filter(lambda x: x != concept, related_concepts))

    return related_concepts

# List_Related_Concepts('安全带','可以-保护人')


# 易混淆的语义特征
# 给到（概念，语义特征）
#   选出所有的同类概念，选出所有同类语义特征，计算每个同类语义特征在该类概念中的线索度
#     按照线索度高低排序
#         如果该特征不为该概念所有
#             则作为混淆项

def List_Confused_Feature(concept, feature):
    # 改成同Subcategory
    sub_category = df_complete[df_complete['Concept'] == concept]['下级类别'].unique()[0]
    related_concepts = list(df_complete[df_complete['下级类别'] == sub_category]['Concept'].unique())
    related_concepts = list(filter(lambda x: x != concept, related_concepts))

    # related_concepts = List_Related_Concepts(concept,feature)
    df_related = df_complete[(df_complete['Concept'].isin(related_concepts))]

    relationship = df_complete[(df_complete['Concept'] == concept) & (df_complete['Feature'] == feature)]['Rel'].unique()[0]
    confused_feature_list = list(df_related[df_related['Rel'] == relationship].sort_values(by=['categorical_distinctiveness','cue_validity'],ascending=False)['Feature'].unique())
    confused_feature_list = list(filter(lambda x: x not in list(df_complete[df_complete['Concept'] == concept]['Feature']), confused_feature_list))

    return confused_feature_list

# List_Confused_Feature('安全带','可以-保护人')[0:10]


In [16]:
## Input 
word_list = ['轿车',
  '摩托车',
  '消防车',
  '消防栓',
  '水',
  '冰红茶',
  '谷子',
  '薄荷',
  '白菜']
# word_list = cue_words


data = {
 "steps": []
}
for word in word_list:
    new_step = {
        'result':'',
        'status':'unchecked',
        "graph": {
            "id": "root",
            "label": word,
            'img':'/SemanticNetwork_DataCollectionTools/test_images/'+word+'.jpeg',
            # 'img':'/test_images/'+word+'.jpeg',

            "children": []
        },
        "mention": {
            "属于":[],
            "用于":[],
            "做":[],
            "有":[],
            "在":[],
            "联想到":[],
        }
    }


    concept = word
    maincategory = df_complete[df_complete['Concept'] == concept]['上级类别'].unique()[0]
    mention_belongto = list(df_complete[df_complete['上级类别'] == maincategory]['下级类别'].unique())[0:3]
    # rel_list = list(df_complete['Rel'].unique())

    feature_function = df_complete[(df_complete['Concept']==concept)&(df_complete['Rel']=='可以')].sort_values(by=['cue_validity'],ascending=False)['Feature'].unique()[0]
    mention_function = List_Confused_Feature(concept,feature_function)[0:3]
    mention_function.append(feature_function)

    feature_need = df_complete[(df_complete['Concept']==concept)&(df_complete['Rel']=='需要')].sort_values(by=['cue_validity'],ascending=False)['Feature'].unique()[0]
    mention_need = List_Confused_Feature(concept,feature_need)[0:3]
    mention_need.append(feature_need)

    feature_have = df_complete[(df_complete['Concept']==concept)&(df_complete['Rel']=='有')].sort_values(by=['cue_validity'],ascending=False)['Feature'].unique()[0]
    mention_have = List_Confused_Feature(concept,feature_have)[0:3]
    mention_have.append(feature_have)

    feature_similar = df_complete[(df_complete['Concept']==concept)&(df_complete['Rel']=='像')].sort_values(by=['cue_validity'],ascending=False)['Feature'].unique()[0]
    mention_similar = List_Confused_Feature(concept,feature_similar)
    if '像-'+concept in mention_similar:
        mention_similar.remove('像-'+concept)
    mention_similar=mention_similar[0:3]
    mention_similar.append(feature_similar)

    new_step['mention']['属于'] = mention_belongto
    new_step['mention']['用于'] = [i.replace('可以-','')  for i in mention_function]
    new_step['mention']['做'] = [i.replace('需要-','')  for i in mention_need]
    new_step['mention']['有'] = [i.replace('有-','')  for i in mention_have]
    new_step['mention']['联想到'] = [i.replace('像-','')  for i in mention_similar]
    
    data['steps'].append(new_step)


with open("02_VASystemData/test_SFA.json", "w") as f: 
    json.dump(data, f, ensure_ascii=False)